In [1]:
# Importação das bibliotecas necessárias
import pandas as pd

import requests

from pendulum import now

In [2]:
fonte = 'lapig'
dataset_name = 'pastagens_degradacao'
raiz = '/home/eduardo/repositorios/IS_Agro/databases'

ano_anterior = now().year - 1

#### Bronze

In [3]:
# Tabelas relativas ao vigor (degradação) das áreas de pastagem no Brasil

contador = 1

for ano in range(2000, ano_anterior):
    url = f'https://s3.lapig.iesa.ufg.br/ows/country/BRASIL/csv/pasture_vigor_col8_s100/brasil_pasture_vigor_col8_s100_year={ano}.csv'
    response = requests.get(url)
    if response.status_code == 200:
        if contador == 1:
            df_vpastagem = pd.read_csv(url)
            contador += 1
        else:
            df_vpastagem = pd.concat(
                [df_vpastagem,
                 (pd.read_csv(url))],
                 ignore_index=True            
            )
            contador += 1
    else:
        pass

df_vpastagem.to_csv(f'{raiz}/bronze_{fonte}_{dataset_name}.csv')


#### Prata

In [17]:
# Área de pastagens a partir do nível de degradação

df_munic = (pd.read_csv(f'{raiz}/bronze_{fonte}_{dataset_name}.csv',
                        usecols=[2, 7, 11, 13],
                        dtype={'ano': int, 'geocod_mun':int, 'classe': str, 'area_past_ha': float})
                        .rename(columns={
                            'ano': 'data',
                            'geocod_mun': 'geocodigo',
                            'area_past_ha': 'area'
                            })
                        .query('area != 0.0') # Exclusão de possíveis registros que possuam valor zerado
                        .assign(data = lambda df: pd.to_datetime(df.data, format='%Y')) # Conversão de número inteiro para formato de data seguindo a ISO 8601 
                        .sort_values(by=['geocodigo', 'data']) # Organização dos dados
                        .set_index(['geocodigo', 'data', 'classe'])
            )
df_munic.to_parquet(f'{raiz}/prata_{fonte}_{dataset_name}_municipio.parquet')

df_uf = (df_munic
         .reset_index()
         .assign(geocodigo_uf = lambda x: x.geocodigo.astype(str).str[:2].astype(int))
         .groupby(['geocodigo_uf', 'data', 'classe'])
         .agg({'area':'sum'})
         .rename_axis(index={'geocodigo_uf': 'geocodigo'})
         .convert_dtypes()
        )
df_uf.to_parquet(f'{raiz}/prata_{fonte}_{dataset_name}_uf.parquet')

### Pastagem

In [19]:
dataset_name = 'pastagem'

#### Bronze

In [8]:
# Tabelas relativas as áreas de pastagem no Brasil

contador = 1

for ano in range(2000, ano_anterior):
    url = f'https://s3.lapig.iesa.ufg.br/ows/country/BRASIL/csv/pasture_col8_s100/brasil_pasture_col8_s100_year={ano}.csv'
    response = requests.get(url)
    if response.status_code == 200:
        if contador == 1:
            df_pastagem = pd.read_csv(url)
            contador += 1
        else:
            df_pastagem = pd.concat(
                [df_pastagem,
                 (pd.read_csv(url))],
                 ignore_index=True            
            )
            contador += 1
    else:
        pass

df_pastagem.to_csv(f'{raiz}/bronze_{fonte}_{dataset_name}.csv')

#### Prata

In [20]:
# Área de pastagens a partir do nível de degradação

df_p_munic = (pd.read_csv(f'{raiz}/bronze_{fonte}_{dataset_name}.csv',
                        usecols=[2, 8, 12],
                        dtype={'ano': int, 'geocod_mun':int, 'area_past_ha': float})
                        .rename(columns={
                            'ano': 'data',
                            'geocod_mun': 'geocodigo',
                            'area_past_ha': 'area'
                            })
                        .query('area != 0.0') # Exclusão de possíveis registros que possuam valor zerado
                        .assign(data = lambda df: pd.to_datetime(df.data, format='%Y')) # Conversão de número inteiro para formato de data seguindo a ISO 8601 
                        .sort_values(by=['geocodigo', 'data']) # Organização dos dados
                        .set_index(['geocodigo', 'data'])
            )
df_p_munic.to_parquet(f'{raiz}/prata_{fonte}_{dataset_name}_municipio.parquet')

df_p_uf = (df_p_munic
         .reset_index()
         .assign(geocodigo_uf = lambda x: x.geocodigo.astype(str).str[:2].astype(int))
         .groupby(['geocodigo_uf', 'data'])
         .agg({'area':'sum'})
         .rename_axis(index={'geocodigo_uf': 'geocodigo'})
         .convert_dtypes()
        )
df_p_uf.to_parquet(f'{raiz}/prata_{fonte}_{dataset_name}_uf.parquet')